## Advanced Tuning of Parameters
#### Using SciKit-Learn Pipelines

Walkthrough by: Samson Bakos

Pipelines in Scikit-Learn provide a convenient interface for executing a sequence of machine learning tasks, which can include data normalization, scaling, dimensionality reduction, and modeling. They allow us to create a streamlined, reproducible object capable of taking raw input data and producing prediction outputs.

Additionally, pipelines prevent data leakage by performing fitting steps only on the training data. They also allow us to optimize hyperparameters across the entire pipeline, rather than just focusing on those of the ML model.

Pipelines offer a structured and automated approach to model development, and are a popular choice for model deployment.

In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
data = fetch_california_housing()
X_train, X_test, y_train, y_test = train_test_split(data['data'], data['target'])

The [California Housing Dataset](https://scikit-learn.org/stable/datasets/real_world.html#california-housing-dataset) includes 8 features for 20640 examples, with each example being a census block group (an area with a typical population of 600-3000). The target is median value of a house in the area in hundreds of thousands of dollars. 

Read more about the data at the link above.

Lets import what we'll need to start this tutorial: 

In [2]:
from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer
from sklearn.feature_selection import SelectKBest, f_regression, RFECV
from sklearn.linear_model import Ridge

We are going to do the following:

1. Scale our data with one of the following methods:
    - [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html), which subtracts the mean and scales by the std. deviation
    - [RobustScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html), similar to StandardScaler, but subtracts the median and scales by the interquartile range (75th Quantile - 25th quantile), allowing it to be less sensitive to outliers
    - [QuantileTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.QuantileTransformer.html), which maps the data to a uniform distribution between 0 and 1, effectively transforming the value of each data point correspond to its quantile in the overall distribution. I.e. the datapoint representing the original 80th quantile would have a transformed value of 0.80

2. Reduce the dimensionality of our data with one of the following methods
    - [SelectKBest](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html), which selects the `K` best features based on a scoring function. In this case, we will use [f_regression](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_regression.html#sklearn.feature_selection.f_regression), which effectively ranks our numeric features by how correlated they are with the target. Ultimately, we will select the `K` most correlated features. 
    - [RFECV](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFECV.html). This is a more sophisticated approach that fits a series of models on the features, starting from the full set of features, each time eliminating the feature with the lowest prediction importance in the model. With RFECV there is no need to tune  a number of features `K`, because the score of each feature subset is determined by cross validation, and the algorithm will stop when it is no longer able to improve CV score by eliminating features. Note that this method is more computationally intensive, as it is effectively cross validation within cross validation, similar to hyperparameter tuning.

3. Perform regression using a simple regularized linear model ([Ridge](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html)). This approach can easily be extended to other sklearn estimators, including classifiers with some tweaks.

## Manual Building Without Pipelines

We'll start by setting things up manually, to highlight how repetitive performing a full machine learning workflow can be without pipelines

First, we need to intialize all of our objects

In [3]:
scaler = StandardScaler()
selector = RFECV(estimator= Ridge())
ridge = Ridge()

Then we will chain the steps in a pipeline like approach

In [4]:
X_train_transformed = scaler.fit_transform(X_train)
X_train_transformed = selector.fit_transform(X_train_transformed, y_train)
ridge.fit(X_train_transformed, y_train); 

Of course, if we want to score on my test data, we need to repeat the preprocessing steps. We also need to be very careful to transform the test data, but NOT refit it.

In [5]:
# # This won't work, because the Ridge model is expecting less features. 
# # Even if it did, the score would be wrong, because we also havent scaled our features

# ridge.score(X_test, y_test) 

In [6]:
X_test_transformed = scaler.transform(X_test)
X_test_transformed = selector.transform(X_test_transformed)
print('Testing score: ', round(ridge.score(X_test_transformed, y_test),3))

Testing score:  0.618


This is kind of repetitive, and it requires us to be very careful to not leak information. Also, if we wanted to make upstream changes, like trying a different scaling algorithm, we would have to redo the whole process!

## Building a Pipeline

Now we'll build a single object to handle the entire workflow

In [7]:
from sklearn.pipeline import Pipeline
pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('reduce_dim', RFECV(estimator= Ridge())),
        ('regressor', Ridge())
        ])

The pipeline is just a list of ordered elements, each with a name and a corresponding object instance. The pipeline module leverages the common interface that every scikit-learn library must implement, such as: `fit` and `transform`/`predict`.

Now, we can fit and score our pipeline extremely quickly with just a few commands:

In [8]:
pipe = pipe.fit(X_train, y_train)
print('Testing score: ', round(pipe.score(X_test, y_test),3))

Testing score:  0.618


The pipeline is just a wrapper around our objects, like a list storing multiple elements.

We can still access individual elements by indexing the pipeline.

For example, what if we want to access the slope and intercept of our ridge model?

In [9]:
print(pipe.steps[2]) # the 3rd element of of the pipe is our ML Model

# But this stores both the label and the object, so we need to index one deeper

('regressor', Ridge())


In [10]:
print(pipe.steps[2][1].coef_)
print(pipe.steps[2][1].intercept_)

[ 0.83547596  0.11883801 -0.30082719  0.34503604 -0.0043885  -0.04523799
 -0.89529611 -0.86636514]
2.0782362661494784


Alteratively we can use the label, kind of like the key in a dictionary

In [11]:
print(pipe['regressor'].intercept_)

2.0782362661494784


This image from [Sebastian Rashcka](https://web.archive.org/web/20210514173616/https://nbviewer.jupyter.org/github/rasbt/python-machine-learning-book/blob/master/code/ch06/ch06.ipynb#Combining-transformers-and-estimators-in-a-pipeline) shows us the intuition of the flow of steps in a pipeline. 

When we call `.fit()` on our pipeline with our training data (left side of image), it applies the `.fit_transform()` methods for our prepreprocessing steps, and `.fit()` on our estimator. 

When we call `.score()` or `.predict()` on our pipeline with our evaluation data (right side of image), it applies `.transform()` for the preprocessing, and either scores or predicts, depending on what we specified.

We don't need to be worried about information leakage in our process, as long as we are careful to supply only training data to `.fit()`.

![](img/pipelineflow.png)

## Tuning Hyperparameters in a Pipeline (Basic)

Pipeline objects also allow us to tune hyperparamters across our entire workflow. We already have experience tuning the hyperparameters of our model objects, but thats not the only place we have hyperparameters. If we use `SelectKBest` as our feature selection, that requires us to select a number of features to keep, which is a hyperparameter.

Its important to note that the hyperparameters of our preprocessing steps are NOT independent from the hyperparameters of our modelling algorithm - depending on how we set up our estimator, there may be a different optimal setting of `K`.

For example, if $\alpha$ in our Ridge regression is high, meaning a high degree of regularization, we may not be as incentivized to reduce the number of features since the model will be less prone to overfitting to noise in the features.

Pipeline objects allow us to tune our entire workflow at once, so we can select the best overall setting for our prediction.

In [12]:
pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('reduce_dim', SelectKBest(f_regression)), # using SelectKBest so we have a k to tune
        ('regressor', Ridge())
        ])

In [13]:
import numpy as np
k_feat = np.arange(1, 9) # 1-8 features 
alpha_to_test = 10.0**np.arange(-6, +6) #alpha from 10^-6 to 10^5

Notice that we use the 'double under' or 'dunder' naming convention. We specify the name of our step in the pipeline, like `reduce_dim`, then to specify which hyperparameter of this object we are optimizing, we give two underscores `__`, then the name of the argument `k`. So all together `'reduce_dim__k'` is the number of features in the 'reduce_dim' step of our pipeline

In [14]:
params = {'reduce_dim__k': k_feat,
              'regressor__alpha': alpha_to_test}

Now we can grid search over these options to find the best overall hyperparameters

In [15]:
from sklearn.model_selection import GridSearchCV
gridsearch = GridSearchCV(pipe, params, verbose=1).fit(X_train, y_train)
print('Final score is: ', gridsearch.score(X_test, y_test), '\nBest Params are: ', gridsearch.best_params_)

Fitting 5 folds for each of 96 candidates, totalling 480 fits


Final score is:  0.6183572155923449 
Best Params are:  {'reduce_dim__k': 8, 'regressor__alpha': 10.0}


## Tuning Hyperparameters in a Pipeline - Selecting Methods

In addition to selecting the best numerical settings for a chosen set of methods, we can also use pipelines to choose between multiple different methods for each step.

So far we've used both `SelectKBest` and `RFECV` for feature selection, but we haven't determined which is better. Admittedly they might be the same for this dataset since its only 8 features, and our previous `SelectKBest` outputted 8, but we don't know for sure if thats still going to be best if we change other parts of the pipeline - i.e. what if we changed the estimator? 

We've also just been using `StandardScaler`, but we have other options available. The [dataset documentation](https://scikit-learn.org/stable/datasets/real_world.html#california-housing-dataset) makes specific reference to possible outliers: "Since the average number of rooms and bedrooms in this dataset are provided per household, these columns may take surprisingly large values for block groups with few households and many empty houses, such as vacation resorts." 

We may be better off with a scaler that is less sensitive to outliers, but we'll have to test to know for sure!

In [16]:
scalers_to_test = [StandardScaler(), RobustScaler(), QuantileTransformer()]
selectors_to_test = [RFECV(estimator= Ridge()), SelectKBest(f_regression, k = 8)] 

Now, we're going to treat choosing which algorithms to use as a hyperparameter, and follow the same intuition as we did when gridsearching for our numerical hyperparams

Because the `scaler` and `reduce_dim` parts of our `params` object are referring to the whole object to be used, not a hyperparameter of the object (like $\alpha$ in Ridge), we don't need the `__` notation. We'll keep it for optimizing $\alpha$ though.

In [17]:
params = {'scaler': scalers_to_test,
        'reduce_dim': selectors_to_test,
        'regressor__alpha': alpha_to_test}


You may have noticed we set `k=8` in the `SelectKBest` above. We already tested this, and know that `k=8` is probably best, but this won't always be the case, so we may want to try both different methods AND different settings for those methods. But we have a problem, because if I want to specify `reduce_dim__k`, the algorithm will break when it tries to apply this to `RFECV`. `RFECV` doesn't have an argument `k`, so passing it this argument will cause an error.

The commented code below will throw an error for this reason.

In [18]:
# params = {'scaler': scalers_to_test,
#         'reduce_dim': selectors_to_test,
#         'reduce_dim__k': k_feat, 
#         'regressor__alpha': alpha_to_test}

# gridsearch = GridSearchCV(pipe, params, verbose=1).fit(X_train, y_train)
# print('Final score is: ', gridsearch.score(X_test, y_test), '\nBest Params are: ', gridsearch.best_params_)

Thankfully, our gridsearch can handle a list of dictionaries as possible options to address this! Our gridsearch will check all options available in the first dictionary, before moving on to the second.

In [19]:
params = [
        # all scalers and alphas with RFECV
        {'scaler': scalers_to_test,
         'reduce_dim': [RFECV(estimator= Ridge())],
         'regressor__alpha': alpha_to_test},

        # all scalers, alphas, and k's with SelectKBest    
        {'scaler': scalers_to_test,
         'reduce_dim': [SelectKBest(f_regression)],
         'reduce_dim__k': k_feat,
         'regressor__alpha': alpha_to_test}
        ]

And we can again do our grid search. A little slower now because we're trying a lot more things

In [20]:
gridsearch = GridSearchCV(pipe, params, verbose=1).fit(X_train, y_train)
print('Final score is: ', gridsearch.score(X_test, y_test), '\nBest Params are: ', gridsearch.best_params_)

Fitting 5 folds for each of 324 candidates, totalling 1620 fits


Final score is:  0.6183572155923449 
Best Params are:  {'reduce_dim': SelectKBest(k=8, score_func=<function f_regression at 0x12c9f56c0>), 'reduce_dim__k': 8, 'regressor__alpha': 10.0, 'scaler': StandardScaler()}


It looks like our original tuned pipeline with `StandardScaler`, `SelectKBest(k = 8)` and `Ridge(alpha = 10)` was the best option. But its certainly good to check!

We can also apply this exact same approach to model selection by passing a list of options to our `'regressor'` step, i.e. we could try `Ridge`, `Lasso`, `RandomForest`, `SVR`, etc. However, adding more and more layers to be simultaneously tuned, each with their own unique hyperparameters, can become unwieldy extremely quickly in terms of both coding our parameter grid, and total computation time.

One way to control the computation time is to use a `RandomSearchCV`, and set the number of iterations - though in this case we are no longer guaranteed to find the best possible setting for the provided options. This is especially relevant for larger datasets and more complex models, where each hyperparameter fit takes longer. We can already see that our final grid search is starting to take some time even for a (relatively) small dataset, with a very simple estimator (Ridge).

It is still probably best practice to give each estimator its own pipeline, to better keep track of things and keep our code more manageable, especially if we are also fitting different methods for feature selection and scaling for each estimator.

For example, if we also wanted to test a more complex model like a Random Forest, we could create another pipeline for this model, and limit the computation with the `n_iter` argument of our `RandomizedSearchCV`.

**Note**: this will still take a while (9.5 minutes on my 2022 MacBook)! You can reduce `n_iter` if you just want to see it run quickly, but the score might not be as good.

In [24]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

max_depths = [5, 10, 15, 20, None]

params = [
        # all scalers and max depth with RFECV
        {'scaler': scalers_to_test,
         'reduce_dim': [RFECV(estimator= Ridge())],
         'regressor__max_depth': max_depths},

        # all scalers, alphas, and k's with SelectKBest    
        {'scaler': scalers_to_test,
         'reduce_dim': [SelectKBest(f_regression)],
         'reduce_dim__k': k_feat,
         'regressor__max_depth': max_depths}
]

We didn't discuss this before because we had already defined a pipe object, but if we're selecting between multiple methods for a step in our param grid, it doesn't actually matter what we start with in our pipeline.

In [27]:
pipe = Pipeline([
        ('scaler', None), # default to none, it will pick from our options in the grid
        ('reduce_dim', None), # same as above
        ('regressor', RandomForestRegressor())
        ])

random_search = RandomizedSearchCV(pipe, params, verbose=1, n_iter = 50).fit(X_train, y_train)
print('Final score is: ', random_search.score(X_test, y_test), '\nBest Params are: ', random_search.best_params_)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Final score is:  0.8083890989653187 
Best Params are:  {'scaler': StandardScaler(), 'regressor__max_depth': 20, 'reduce_dim': RFECV(estimator=Ridge())}
